In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import scanpy as sc
import scipy as sp

import sys
import os
sys.path.append(os.path.abspath("./utility_functions"))

import rz_functions as rz
import rz_utility_spring as srz

from time import time

/home/ng136/miniconda3/envs/py36/lib/python3.6/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


python version: 3.6.10


In [2]:
project_dir = 'spring/tox_marie/'
plot_name = 'all_cells_200406_04h07'

In [3]:
# get cell filter:
cell_ix = np.loadtxt(project_dir+plot_name+'/cell_filter.txt',dtype=int)

# gene list
gene_list = np.loadtxt(project_dir+'/genes.txt',dtype=str)

In [2]:
adata = sc.read_h5ad('backups/tox_marie_all_cells_unnormalized_pca_umap_leiden_35358x22631_backup_200512_04h53.h5ad')

In [3]:
Eraw = adata.X

In [4]:
Eraw.shape

(35358, 22631)

# Load reference expression profiles in an AnnData object compatible with scanpy¶

In [5]:
prepath = 'Zilionis_Immunity_2019/'
# load count data
rzdata = sc.read_mtx(prepath+'GSE127465_mouse_counts_normalized_15939x28205.mtx.gz')

# load cell annotations
rzdata.obs = pd.read_csv(prepath+'GSE127465_mouse_cell_metadata_15939x12.tsv.gz',
            sep='\t',comment='#')

# load gene names
rzdata.var_names = np.loadtxt(prepath+'GSE127465_gene_names_mouse_28205.tsv.gz',dtype=str)

rzdata.shape

(15939, 28205)

In [6]:
#normalize immunity paper data to 10,000 counts per cell
sc.pp.normalize_per_cell(rzdata,counts_per_cell_after=1e4)


Transforming to str index.


In [7]:
cat0 = rz.centroids('Minor subset', rzdata).T

In [8]:
cat0.head()

,Mac1,DC1,DC3,Mono3,Mac2,N4,T2,Mono1,T3,T1,...,Mac3,pDC,NK cells,MonoDC,B cells,Mono2,N2,Mac4,Basophils,N3
0610007P14Rik,0.528133,0.260421,0.469878,0.187770,0.469109,0.190145,0.402685,0.264594,1.063999,0.474985,...,0.157427,1.194086,0.468520,0.475552,0.368709,0.374221,0.083965,0.785277,0.0,0.042854
0610009B22Rik,0.167399,0.544554,0.033153,0.263399,0.104697,0.154510,0.032572,0.099438,0.000000,0.030961,...,0.000000,0.274819,0.141834,0.382270,0.071297,0.138071,0.033403,0.155395,0.0,0.046007
0610009L18Rik,0.000000,0.000000,0.020329,0.011041,0.000000,0.030162,0.118733,0.007362,0.000000,0.026514,...,0.030877,0.075785,0.017998,0.000000,0.013612,0.010631,0.000000,0.000000,0.0,0.000000
0610009O20Rik,0.020461,0.027731,0.015273,0.065693,0.003815,0.029087,0.101459,0.049431,0.073788,0.038778,...,0.000000,0.150212,0.054022,0.000000,0.062836,0.000000,0.000000,0.059979,0.0,0.103472
0610010F05Rik,0.122463,0.015066,0.272357,0.025895,0.066593,0.058477,0.109913,0.078456,0.244052,0.045135,...,0.000000,0.058566,0.111800,0.078463,0.038325,0.084380,0.064667,0.116328,0.0,0.000000


In [11]:
pseudo = 0.1

cat = cat0 + pseudo

# Find common genes between the two datasets.
 Filtering on variable genes is also a possibility but I start by simply using all genes

In [12]:
Eraw = adata.X
print(type(Eraw))

gene_list = adata.var_names

<class 'scipy.sparse.csr.csr_matrix'>


In [13]:
# common genes
gmask = np.in1d(gene_list, cat.index)

# genes detected in the current dataset:
m2 = np.array(Eraw.sum(axis=0))[0]>0

# combine masks
gmask = gmask&m2


common_genes = gene_list[gmask]
print(len(gene_list),len(cat.index),len(common_genes))

22631 28205 15366


In [14]:
print(Eraw.shape)

(35358, 22631)


In [15]:
start = time()
bays = []
i = 0
step=5000
comment = 'tox_marie_all_cells'
for j in range(step,Eraw.shape[0]+step,step):
    
    # Eraw - sparse cells x gene matrix
    j = min(j,Eraw.shape[0])
    tmp_dense = pd.DataFrame(Eraw.T[gmask][:,i:j].todense())
    tmp_dense.index = np.array(gene_list)[gmask]
    
    bay = rz.bayesian_classifier(tmp_dense,cat.loc[common_genes])
    bays.append(bay)
    i0 = i
    i = j
    
    print('%.2f min.'%((time()-start)/60.))
    print('cells from %d to %d done'%(i0,j))

# conenate
bay = pd.concat(bays,axis=1)

# reset index
bay.columns = np.arange(bay.shape[1])

fname = 'backups/loglikelihoods_bay_classif_%s_%s'%(comment,rz.now())
print(fname)
rz.save_df(bay,fname)

1.59 min.
cells from 0 to 5000 done
3.13 min.
cells from 5000 to 10000 done
4.67 min.
cells from 10000 to 15000 done
6.22 min.
cells from 15000 to 20000 done
7.76 min.
cells from 20000 to 25000 done
9.30 min.
cells from 25000 to 30000 done
11.06 min.
cells from 30000 to 35000 done
11.17 min.
cells from 35000 to 35358 done
backups/loglikelihoods_bay_classif_tox_marie_all_cells_200423_20h42


# Add results to SPRING plot (to do)

In [16]:
# get profile with max likelihood
mostlikely = bay.idxmax().values
print(len(mostlikely))
print(len(cell_ix))

# use cell_ix to only select cell in the desired spring plot:
mostlikely = list(mostlikely[cell_ix])
mostlikely[:10]

35358
35358


['Mac1', 'N1', 'NK cells', 'Mono2', 'T1', 'DC3', 'N2', 'T2', 'N2', 'Mac1']

In [17]:
bay.head()

,0,1,2,3,4,5,6,7,8,9,...,35348,35349,35350,35351,35352,35353,35354,35355,35356,35357
Mac1,-39162.118609,-35145.387562,-36759.675174,-35462.852727,-33972.462772,-35021.474375,-34209.800299,-35075.359652,-34648.645732,-39511.479141,...,-35345.142965,-34972.453339,-34339.130683,-33725.252254,-35896.214822,-34045.815666,-34137.365587,-34118.731282,-35206.792544,-33647.921754
DC1,-39654.735740,-37066.770842,-36629.516231,-35277.164524,-33085.122544,-34450.321539,-36381.763101,-34885.443120,-36538.150276,-39904.965410,...,-35262.471926,-34448.416716,-34110.254412,-33052.088903,-35351.116293,-33712.150361,-33550.220790,-33521.677814,-34977.074089,-34959.417713
DC3,-39901.974412,-36659.554409,-36671.233271,-35621.928215,-33432.305634,-32243.555812,-36088.618991,-34837.220525,-36108.491045,-39936.644769,...,-33075.439184,-31880.416581,-32997.966511,-31530.042341,-35906.551468,-32173.910758,-31358.925480,-31803.577192,-32645.884030,-35141.939812
Mono3,-39489.135998,-32177.351615,-37070.546677,-34755.618977,-34604.391279,-35435.119870,-32269.952276,-35020.882318,-32711.302564,-39768.868313,...,-35569.881640,-35552.783070,-34595.055525,-34143.487760,-36472.894296,-34729.953309,-35106.687998,-34677.490725,-35677.095727,-34121.820135
Mac2,-39761.288567,-36467.563246,-37660.605445,-36375.755703,-34940.940318,-36333.917906,-35284.645106,-35986.384228,-35539.409979,-40108.496682,...,-36665.294532,-36391.976465,-35462.274391,-34922.868548,-36637.906763,-35287.929944,-35624.734454,-35241.525054,-36378.381305,-34431.154615
